In [1]:
# pip show gurobipy

Name: gurobipy
Version: 9.5.1
Summary: Python interface to Gurobi
Home-page: https://www.gurobi.com
Author: Gurobi Optimization, LLC
Author-email: 
License: Proprietary
Location: c:\users\daiki\appdata\local\packages\pythonsoftwarefoundation.python.3.9_qbz5n2kfra8p0\localcache\local-packages\python39\site-packages
Requires: 
Required-by: gurobipy-stubs
Note: you may need to restart the kernel to use updated packages.


In [1]:
from gurobipy import *

## 問題例1 モデルの変更

In [37]:
# nと初期配置を変更しなければいけない
ship_num=4
stack_num=3
height=4
time = 10

O1=[i+1 for i in range(ship_num)]

S=[i+1 for i in range(stack_num)]
H=[i+1 for i in range(height)]
P=[i+1 for i in range(ship_num)]
T=[i+1 for i in range(time)]
n1=[1,2,4,3]
# n1=[n_[O1[0]-1],n_[O1[1]-1],n_[O1[2]-1],n_[O1[3]-1]]

# S,H,P,T,n1,O

In [38]:
from asyncio.constants import SENDFILE_FALLBACK_READBUFFER_SIZE
import random
from random import seed
import numpy as np
from scipy.stats import multivariate_normal

np.random.seed(0)

# 期待値と分散共分散行列の準備
mean = np.arange(1,2*ship_num,2)
cov=np.zeros((ship_num,ship_num))
for i in range(ship_num):
  for j in range(ship_num):
    if i==j:
      cov[i][i]=4*np.random.randn()
    elif i>j:
      cov[i][j]=np.random.randn()
      cov[j][i]=cov[i][j]

# numpy を用いた生成
size=10
data_1 = np.random.multivariate_normal(mean, cov, size=size)

# print(data_1.shape)

# print(np.mean(data_1, axis=0))

# print(np.cov(data_1, rowvar=False))

# print(data_1)
O_=np.argsort(data_1)
O_=O_+np.ones((size,ship_num)).astype(int)
O=O_.tolist()

# data_2 = multivariate_normal(mean, cov).rvs(size=200)

# print(data_2.shape)

# print(np.mean(data_2, axis=0))

# print(np.cov(data_2, rowvar=False))

C:\Users\daiki\AppData\Local\Temp\ipykernel_4296\239299481.py:22: RuntimeWarning: covariance is not positive-semidefinite.
  data_1 = np.random.multivariate_normal(mean, cov, size=size)


In [39]:
a=[]
d={}
for i in O:
  # print(i)
  if not i in a:
    a.append(i)
for j in a:
  d[tuple(j)]=O.count(j)

d


{(1, 3, 2, 4): 3,
 (2, 1, 3, 4): 2,
 (1, 4, 2, 3): 1,
 (1, 2, 3, 4): 1,
 (1, 4, 3, 2): 1,
 (3, 2, 1, 4): 1,
 (2, 3, 1, 4): 1}

In [40]:
m=Model("IP6")

# alpha=m.addVar(vtype="C")

# 変数の定義
x,y,z,w={},{},{},{}
for s in S:
  for h in H:
    for p in P:
      for t in T:
        x[s,h,p,t]=m.addVar(vtype="B")


for s in S:
  for h in H:
    for p in P:
      for t in T:
        w[s,h,p,t]=m.addVar(vtype="C",lb=0)
        z[s,h,p,t]=m.addVar(vtype="C",lb=0)


# for s in S:
#   for h in range(2,len(H)+1):
#     for i,o in enumerate(O):
#       z[s,h,i]=m.addVar(vtype="c",lb=0)

# for i in range(len(O)):
#   w[i]=m.addVar(vtype="C",lb=0)

# 制約条件
for p in P:
   for t in range(1,len(T)):
      m.addConstr(quicksum(w[s,h,p,t] for s in S for h in H)==quicksum(z[s,h,p,t] for s in S for h in H))

for t in range(1,len(T)):
   m.addConstr(quicksum(w[s,h,p,t] for s in S for h in H for p in P)<=1)
   m.addConstr(quicksum(z[s,h,p,t] for s in S for h in H for p in P)<=1)

for t in range(1,len(T)-1):
  m.addConstr(quicksum(w[s,h,p,t+1] for s in S for h in H for p in P)<=quicksum(w[s,h,p,t] for s in S for h in H for p in P))

for s in S:
  for t in range(2,len(T)+1):
    m.addConstr(quicksum(x[s,1,p,t] for p in P)+quicksum(z[s,1,p,t] for p in P)<=1)

for s in S:
  for h in range(1,len(H)):
    for t in range(1,len(T)+1):
      m.addConstr(quicksum(x[s,h+1,p,t] for p in P)+quicksum(w[s,h,p,t] for p in P)+quicksum(z[s,h+1,p,t] for p in P)<=quicksum(x[s,h,p,t] for p in P))

for s in S:
  for h in H:
    for p in P:
      for t in range(1,len(T)):
        m.addConstr(x[s,h,p,t]+z[s,h,p,t]==x[s,h,p,t+1]+w[s,h,p,t])

for s in S:
  for h in range(1,len(H)):
    for t in range(1,len(T)):
      m.addConstr(quicksum(x[s,h,p,t] for p in P)+quicksum(x[s,h+1,p,t] for p in P)+quicksum(w[s,h,p,t] for p in P)+quicksum(z[s,h+1,p,t] for p in P)+quicksum(z[s,h,p,t] for p in P)<=2)

for s in S:
  for t in range(1,len(T)-1):
    m.addConstr(quicksum(z[s,h,p,t] for h in H for p in P)+quicksum(w[s,h,p,t+1] for h in H for p in P)<=1)

for s in S:
  for p in P:
    for t in range(1,len(T)-1):
      m.addConstr(quicksum(w[s,h,p,t] for h in H)+quicksum(z[s,h,p,t+1] for h in H)<=1)

for s in S:
  for h in range(1,len(H)):
    for p in P:
      m.addConstr(quicksum(x[s,h+1,k,len(T)] for k in range(p,len(P)+1))<=quicksum(x[s,h,k,len(T)] for k in range(p,len(P)+1)))

for s in S:
  for h in H:
    m.addConstr(w[s,h,1,len(T)-1]==0)

for s in S:
  for p in range(2,len(P)+1):
    m.addConstr(w[s,1,p,len(T)-1]==0)

for s in S:
  for h in H:
    for p in P:
      m.addConstr(z[s,h,p,len(T)]==0)
      m.addConstr(w[s,h,p,len(T)]==0)


# for s in S:
#   for h in range(2,len(H)+1):
#     for i,o in enumerate(O):
#       for j,p in enumerate(o):
#         m.addConstr(z[s,h,i]>=quicksum(x[s,h,k,len(T)] for k in o[j:])-quicksum(x[s,h-1,k,len(T)] for k in o[j:]))

# for i in range(len(O)):
#   m.addConstr(w[i]>=quicksum(z[s,h,i] for s in S for h in H if h!=1)-alpha)

In [41]:
# 初期配置の制約

m.addConstr(x[1,1,4,1]==1)
m.addConstr(x[1,2,4,1]==1)
m.addConstr(x[1,3,2,1]==1)
m.addConstr(x[1,4,3,1]==1)
m.addConstr(x[2,1,4,1]==1)
m.addConstr(x[2,2,2,1]==1)
m.addConstr(x[2,3,3,1]==1)
m.addConstr(x[2,4,1,1]==1)
m.addConstr(x[3,1,2,1]==1)
m.addConstr(x[3,2,2,1]==1)
# m.addConstr(x[3,3,O1[4],1]==1)
# m.addConstr(x[3,4,O1[2],1]==1)
# m.addConstr(x[3,5,O1[0],1]==1)
# m.addConstr(x[4,1,O1[7],1]==1)
# m.addConstr(x[4,2,O1[4],1]==1)
# m.addConstr(x[4,3,O1[0],1]==1)
# m.addConstr(x[4,4,O1[2],1]==1)
# m.addConstr(x[5,1,O1[6],1]==1)
# m.addConstr(x[5,2,O1[1],1]==1)
# m.addConstr(x[6,1,O1[7],1]==1)
# m.addConstr(x[6,2,O1[4],1]==1)
# m.addConstr(x[6,3,O1[6],1]==1)
# m.addConstr(x[6,4,O1[3],1]==1)

<gurobi.Constr *Awaiting Model Update*>

In [42]:
# beta=0.8
# m.setObjective(alpha+10**(-5)*quicksum(y[s,k,t] for s in S for k in S if s!=k for t in range(1,len(T)))+quicksum(w[i] for i in range(len(O)))/((1-beta)*len(O)))
m.setObjective(quicksum(z[s,h,p,t] for s in S for h in H for p in P for t in range(1,len(T))))

In [43]:
m.optimize()
print(m.Status)

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 975 rows, 1440 columns and 9343 nonzeros
Model fingerprint: 0xb9ae1f95
Variable types: 960 continuous, 480 integer (480 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+00]
Presolve removed 348 rows and 552 columns
Presolve time: 0.12s
Presolved: 627 rows, 888 columns, 6167 nonzeros
Variable types: 542 continuous, 346 integer (346 binary)

Root relaxation: objective 2.433962e+00, 735 iterations, 0.09 seconds (0.02 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    2.43396    0  119          -    2.43396      -     -    0s
     0     0    2.75318    0  133          -    2.75318      -     -    0s


In [45]:
EPS=1.e-6

if m.Status == GRB.OPTIMAL:
  # for (s,h,p,t) in x:
  #   if t==1:
  #     if x[s,h,p,t].X>EPS:
  #       print("x[%2s,%2s,%2s,%2s]=%3s" %(s,h,p,t,x[s,h,p,t].X))

  #   if t>=2:
  #     if (x[s,h,p,t].X-x[s,h,p,t-1].X)>EPS:
  #       print("x[%2s,%2s,%2s,%2s]=%3s" %(s,h,p,t,x[s,h,p,t].X))

  for (s,h,p,t) in z:
    if z[s,h,p,t].X>EPS:
      print("z[%2s,%2s,%2s,%2s]=%3s" %(s,h,p,t,z[s,h,p,t].X))

  for (s,h,p,t) in w:
    if w[s,h,p,t].X>EPS:
      print("w[%2s,%2s,%2s,%2s]=%3s" %(s,h,p,t,w[s,h,p,t].X))

  # for (s,h,i) in z:
  #   if z[s,h,i].X>EPS:
  #     print("z[%2s,%2s,%2s]=%3s" %(s,h,i,z[s,h,i].X))
  print("the objective function", m.objVal)
  # print("VaR=",alpha.X)

z[ 1, 3, 3, 4]=1.0
z[ 1, 4, 3, 5]=1.0
z[ 2, 3, 2, 6]=1.0
z[ 2, 4, 3, 2]=1.0
z[ 3, 3, 1, 1]=1.0
z[ 3, 4, 2, 3]=1.0
w[ 1, 3, 2, 3]=1.0
w[ 1, 4, 3, 2]=1.0
w[ 2, 3, 3, 5]=1.0
w[ 2, 4, 1, 1]=1.0
w[ 2, 4, 3, 4]=1.0
w[ 3, 4, 2, 6]=1.0
the objective function 6.0


In [ ]:
O